In [8]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Logical;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

    // Task 2 (2 points). f(x) = 1 if at least two of three input bits are different - easy version
    //
    // Inputs:
    //      1) a 3-qubit array "inputs",
    //      2) a qubit "output".
    // Goal: Implement a marking oracle for function f(x) = 1 if at least two of the three bits of x are different.
    //       That is, if both inputs are in a basis state, flip the state of the output qubit 
    //       if and only if the three bits written in the array "inputs" have both 0 and 1 among them,
    //       and leave the state of the array "inputs" unchanged.
    //       The effect of the oracle on superposition states should be defined by its linearity.
    //       Don't use measurements; the implementation should use only X gates and its controlled variants.
    //       This task will be tested using ToffoliSimulator.
    // 
    // For example, the result of applying the operation to state (|001⟩ + |110⟩ + |111⟩)/√3 ⊗ |0⟩
    // will be 1/√3|001⟩ ⊗ |1⟩ + 1/√3|110⟩ ⊗ |1⟩ + 1/√3|111⟩ ⊗ |0⟩.
    //

In [13]:
  operation Task2_ValidTriangle (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl {
        Controlled X(inputs, output);
        ApplyToEachCA(X, inputs);
        Controlled X(inputs, output);
        X(output);
        ApplyToEachCA(X, inputs);
    }

Task2_ValidTriangle

In [14]:
internal operation VerifySingleOutputFunction(numInputs : Int, op : ((Qubit[], Qubit) => Unit is Adj+Ctl), predicate : (Int -> Bool)) : Unit {
        for assignment in 0 .. 2^numInputs - 1 {
            use (inputs, output) = (Qubit[numInputs], Qubit());
            within {
                ApplyXorInPlace(assignment, LittleEndian(inputs));
                AllowAtMostNCallsCA(0, Measure, "You are not allowed to use measurements");
            } apply {
                op(inputs, output);
            }

            // Check that the result is expected
            let actual = ResultAsBool(MResetZ(output));
            let expected = predicate(assignment);
            Fact(actual == expected,
                $"Oracle evaluation result {actual} does not match expected {expected} for assignment {IntAsBoolArray(assignment, numInputs)}");

            // Check that the inputs were not modified
            Fact(MeasureInteger(LittleEndian(inputs)) == 0, 
                $"The input states were modified for assignment {assignment}");
        }
    }


    // ------------------------------------------------------
    internal function IsTriangleValid (input : Int) : Bool {
        // the triangle is valid if it has at least two different bits (i.e., not all are the same)
        return input > 0 and input < 7;
    }

    //@Test("ToffoliSimulator")
    operation Test2_ValidTriangle() : Unit {
        VerifySingleOutputFunction(3, Task2_ValidTriangle, IsTriangleValid);
    }

VerifySingleOutputFunction, IsTriangleValid, Test2_ValidTriangle

In [16]:
%toffoli Test2_ValidTriangle

Source,Callable
(notebook),VerifySingleOutputFunction
(notebook),Test2_ValidTriangle


Oracle evaluation result True does not match expected False for assignment [False,False,False]
